<a href="https://colab.research.google.com/github/yukinaga/minnano_rl/blob/main/section_3/01_exercise_section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習: SARSAの実装
Section2で暑かったQ学習のコードを、SARSAのコードに変換します。    
Brainクラスの記述を変更し、SARSAを実装した上で動作を確認しましょう。

## ライブラリの導入

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib import animation, rc

## Brainクラス
エージェントの頭脳となるクラスです。  
以下のセルの指定された箇所にコードを追記し、SARSAを実装しましょう。  

Q学習では以下の式が用いられます。  

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \eta\left(R_{t+1}+\gamma \max_{a}Q(s_{t+1}, a) - Q(s_{t}, a_{t})\right) $$

ここで、$a_{t}$は行動、$s_t$は状態、$Q(s_t,a_t) $はQ値、$\eta$は学習係数、$R_{t+1}$は報酬、$\gamma$は割引率になります。  

一方、SARSAでは以下の式が用いられます。。  

$$ Q(s_t,a_t) \leftarrow Q(s_t,a_t) + \eta\left(R_{t+1}+\gamma Q(s_{t+1}, a_{t+1}) - Q(s_{t}, a_{t})\right) $$

Q学習では$t+1$の時刻における最大のQ値を使いますが、SARSAでは$t+1$において実際に選択した行動のQ値を使います。  

In [ ]:
class Brain:
    def __init__(self, n_state, w_y, w_vy, n_action, gamma=0.9, r=0.99, lr=0.01):
        self.n_state = n_state  # 状態の数
        self.w_y = w_y  # 位置の刻み幅
        self.w_vy = w_vy  # 速度の刻み幅
        self.n_action = n_action  # 行動の数

        self.eps = 1.0  # ε
        self.gamma = gamma  # 割引率
        self.r = r  # εの減衰率
        self.lr = lr  # 学習係数

        self.q_table = np.random.rand(n_state*n_state, n_action)  # Q-Table

    def quantize(self, state, n_state, w):  # 状態の値を整数のインデックスに変換
        min = - n_state / 2 * w
        nw = (state - min) / w
        nw = int(nw)
        nw = 0 if nw < 0 else nw
        nw = n_state-1 if nw >= n_state-1 else nw
        return nw

    def train(self, states, next_states, action, next_action, reward, terminal):  # Q-Tableを訓練
        i = self.quantize(states[0], self.n_state, self.w_y)  # 位置のインデックス
        j = self.quantize(states[1], self.n_state, self.w_vy)  # 速度のインデックス
        q = self.q_table[i*self.n_state+j, action]  # 現在のQ値

        next_i = self.quantize(next_states[0], self.n_state, self.w_y)  # 次の位置のインデックス
        next_j = self.quantize(next_states[1], self.n_state, self.w_vy)  # 次の速度のインデックス
        q_next =   # ← ここにコードを追記

        if terminal:
            self.q_table[i*self.n_state+j, action] = q + self.lr*reward  # 終了時は報酬のみ使用
        else:
            self.q_table[i*self.n_state+j, action] = q + self.lr*(reward + self.gamma*q_next - q)  # Q値の更新式

    def get_action(self, states):
        if np.random.rand() < self.eps:  # ランダムな行動
            action = np.random.randint(self.n_action)
        else:  # Q値の高い行動を選択
            i = self.quantize(states[0], self.n_state, self.w_y)
            j = self.quantize(states[1], self.n_state, self.w_vy)
            action = np.argmax(self.q_table[i*self.n_state+j])
        if self.eps > 0.1:  # εの下限
            self.eps *= self.r
        return action

## エージェントのクラス
SARSAに対応するために、1つ先の時間の行動もtrainメソッドに渡すようにします。

In [ ]:
class Agent:
    def __init__(self, v_x, v_y_sigma, v_jump, brain):
        self.v_x = v_x  # x速度
        self.v_y_sigma = v_y_sigma  # y速度、初期値の標準偏差
        self.v_jump = v_jump  # ジャンプ速度
        self.brain = brain
        self.reset()

    def reset(self):
        self.x = -1  # 初期x座標
        self.y = 0  # 初期y座標
        self.v_y = self.v_y_sigma * np.random.randn()  # 初期y速度
        self.action = np.random.randint(2)  # 最初の行動はランダムに

    def step(self, g):  # 時間を1つ進める g:重力加速度
        states = np.array([self.y, self.v_y])
        self.x += self.v_x
        self.y += self.v_y

        reward = 0  # 報酬
        terminal = False  # 終了判定
        if self.x>1.0:
            reward = 1
            terminal = True
        elif self.y<-1.0 or self.y>1.0:
            reward = -1
            terminal = True
        reward = np.array([reward])

        if self.action == 0:
            self.v_y -= g   # 自由落下
        else:
            self.v_y = self.v_jump  # ジャンプ
        next_states = np.array([self.y, self.v_y])
        next_action = self.brain.get_action(next_states)
        self.brain.train(states, next_states, self.action, next_action, reward, terminal)
        self.action = next_action

        if terminal:
            self.reset()

## 環境のクラス

In [ ]:
class Environment:
    def __init__(self, agent, g):
        self.agent = agent
        self.g = g  # 重力加速度

    def step(self):
        self.agent.step(self.g)
        return (self.agent.x, self.agent.y)

## アニメーション

In [ ]:
def animate(environment, interval, frames):
    fig, ax = plt.subplots()
    plt.close()
    ax.set_xlim(( -1, 1))
    ax.set_ylim((-1, 1))
    sc = ax.scatter([], [])

    def plot(data):
        x, y = environment.step()
        sc.set_offsets(np.array([[x, y]]))
        return (sc,)

    return animation.FuncAnimation(fig, plot, interval=interval, frames=frames, blit=True)

## SARSAの実行
SARSAを実行し、動作を確認しましょう。

In [ ]:
n_state = 10
w_y = 0.2
w_vy = 0.2
n_action = 2
brain = Brain(n_state, w_y, w_vy, n_action, r=0.99)  # εが減衰する

v_x = 0.05
v_y_sigma = 0.1
v_jump = 0.2
agent = Agent(v_x, v_y_sigma, v_jump, brain)

g = 0.2
environment = Environment(agent, g)

anim = animate(environment, 50, 1024)
rc("animation", html="jshtml")
anim

# 解答例
以下に解答例を掲載します。

In [ ]:
class Brain:
    def __init__(self, n_state, w_y, w_vy, n_action, gamma=0.9, r=0.99, lr=0.01):
        self.n_state = n_state  # 状態の数
        self.w_y = w_y  # 位置の刻み幅
        self.w_vy = w_vy  # 速度の刻み幅
        self.n_action = n_action  # 行動の数

        self.eps = 1.0  # ε
        self.gamma = gamma  # 割引率
        self.r = r  # εの減衰率
        self.lr = lr  # 学習係数

        self.q_table = np.random.rand(n_state*n_state, n_action)  # Q-Table

    def quantize(self, state, n_state, w):  # 状態の値を整数のインデックスに変換
        min = - n_state / 2 * w
        nw = (state - min) / w
        nw = int(nw)
        nw = 0 if nw < 0 else nw
        nw = n_state-1 if nw >= n_state-1 else nw
        return nw

    def train(self, states, next_states, action, next_action, reward, terminal):  # Q-Tableを訓練
        i = self.quantize(states[0], self.n_state, self.w_y)  # 位置のインデックス
        j = self.quantize(states[1], self.n_state, self.w_vy)  # 速度のインデックス
        q = self.q_table[i*self.n_state+j, action]  # 現在のQ値

        next_i = self.quantize(next_states[0], self.n_state, self.w_y)  # 次の位置のインデックス
        next_j = self.quantize(next_states[1], self.n_state, self.w_vy)  # 次の速度のインデックス
        q_next = self.q_table[next_i*self.n_state+next_j, next_action]  # ← ここにコードを追記

        if terminal:
            self.q_table[i*self.n_state+j, action] = q + self.lr*reward  # 終了時は報酬のみ使用
        else:
            self.q_table[i*self.n_state+j, action] = q + self.lr*(reward + self.gamma*q_next - q)  # Q値の更新式

    def get_action(self, states):
        if np.random.rand() < self.eps:  # ランダムな行動
            action = np.random.randint(self.n_action)
        else:  # Q値の高い行動を選択
            i = self.quantize(states[0], self.n_state, self.w_y)
            j = self.quantize(states[1], self.n_state, self.w_vy)
            action = np.argmax(self.q_table[i*self.n_state+j])
        if self.eps > 0.1:  # εの下限
            self.eps *= self.r
        return action